<a href="https://colab.research.google.com/github/YoonJiHwan98/BigContest_Project/blob/Jihwan/code/Bigcontest_SVM_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# train, val, test split

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/빅콘테스트_2022/data/train_prepared.csv')
test = pd.read_csv('/content/drive/MyDrive/빅콘테스트_2022/data/target_prepared.csv')
y = pd.read_csv('/content/drive/MyDrive/빅콘테스트_2022/data/train_prepared_label.csv')

In [ ]:
X = train.copy()
#y = train['is_applied']

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=190000)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=190000)

# 모델 학습

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

#svc_model = SVC()
#svc_model.fit(X_train,y_train)

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier  # SVC 가 시간이 너무 오래걸려서 bagging 방법을 활용해서 실행 시간을 줄였습니댜. 

estimator = SVC(kernel = 'rbf',C = 10, gamma = 0.1) ## 파라미터 수정 
n_estimators = 3
n_jobs = 1
svc_model = BaggingClassifier(base_estimator=estimator,
                          n_estimators=n_estimators,
                          max_samples=1./n_estimators,
                          n_jobs=n_jobs)
svc_model.fit(X_train,y_train)

In [ ]:
import joblib
joblib.dump(svc_model, "svc_model.pkl") # DIFF
#...
# my_model_loaded = joblib.load("my_model.pkl")

# 모델 평가

In [ ]:
import os

ROOT_DIR = "/content/gdrive/My Drive/빅콘테스트_2022/"
DATA_PATH = ROOT_DIR + "data/"
IMAGE_PATH = ROOT_DIR + "img/"

os.makedirs(DATA_PATH, exist_ok = True)
os.makedirs(IMAGE_PATH, exist_ok = True)

In [ ]:
def save_data(data, file_name, DATA_PATH=DATA_PATH):
  file_path = os.path.join(DATA_PATH, file_name)
  print(file_path)
  return data.to_csv(file_path, index=False)

In [ ]:
ROOT_DIR = '/content/drive/MyDrive/빅콘테스트_2022/'
SCORE_PATH = ROOT_DIR + "score/"

#os.makedirs(SCORE_PATH, exist_ok = True)
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

In [ ]:
# accuracy, precision, recall, f1 score
def model_validation(model, data, label, mode, file_title):
    pred = model.predict(data)
    print('#### ' + mode + ' score ####')
    print(f'accuracy : {accuracy_score(label, pred)}, precision : {precision_score(label, pred)}, recall : {recall_score(label, pred)}, f1 score : {f1_score(label, pred)}')
    df = pd.DataFrame([], columns=['metric', 'score'])
    df.loc[0] = ['accuracy', accuracy_score(label, pred)]
    df.loc[1] = ['precision', precision_score(label, pred)]
    df.loc[2] = ['recall', recall_score(label, pred)]
    df.loc[3] = ['f1 score', f1_score(label, pred)]

    save_data(df, file_title, SCORE_PATH)
    return df


In [ ]:
model_validation(svc_model, X_train, y_train, 'train', 'SVC_train_score.csv')

In [ ]:
model_validation(svc_model, X_val, y_val, 'train', 'SVC_val_score.csv')

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score

# ROC curve, auc score
# proba  clf.predict_proba(X)[:, 1]

def model_auc(model, data, label, mode):

    proba = model.predict_proba(data)[::, 1]

    print('#### ' + mode + ' ROC AUC score ####')
    print(roc_auc_score(label, proba))

    print('#### ' + mode + ' ROC curve plotting####')
    fpr, tpr, _ = metrics.roc_curve(label, proba)

    plt.figure(figsize=(15,15))
    plt.plot(fpr, tpr)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.title(mode + " ROC curve")
    save_fig(mode + " ROC curve")
    plt.show()
    
